In [ ]:
# Automatically reload external modules (see https://ipython.org/ipython-doc/3/config/extensions/autoreload.html for more information).
%load_ext autoreload
%autoreload 2

# Set up system path to include our own "mldog" python package.
import sys
sys.path.append('../src')

In [ ]:
import os
import pandas as pd
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.feature_extraction import MinimalFCParameters

from mldog.util.drill_detect.data import convertDrillTimeSeriesToFeatures

In [ ]:
# Verzeichnis mit Aufzeichnungsunterverzeichnissen eingeben
input_folder = ".../Datenaufzeichnung 1"

input_folders = [
    os.path.join(input_folder, folder) 
    for folder in os.listdir(input_folder) 
    if os.path.isdir(os.path.join(input_folder, folder))
]

In [ ]:
import features_reduced as f

# Path zu data des Projekts eingeben
output_base_dir = ".../team-10/data/"

Altes feature Extraction einfach in for schleife eingesetzt

In [ ]:
for input_folder in input_folders:
    folder_name = os.path.basename(input_folder)
    output_file = os.path.join(output_base_dir, f"{folder_name}_combined_features.csv")
    
    flatten_file_paths = [
        os.path.join(input_folder, filename)
        for filename in os.listdir(input_folder)
    ]
    
    all_features = pd.DataFrame()
    all_measurements = pd.DataFrame()
    parameters = f.features
    
    done = 0
    for file_path in flatten_file_paths:
        if not file_path.endswith("Hz.csv"):
            done += 1
            continue
        
        df = pd.read_csv(file_path)
        
        # Feature Extraction
        print(f"Processed {done}/{len(flatten_file_paths)} in {folder_name}")
        print(f"Processing: {file_path}")
        extracted_features = convertDrillTimeSeriesToFeatures(df, fcParameters=parameters)

        # File name Spalte
        extracted_features["dataFile"] = os.path.basename(file_path)

        # Zu Dataframe hinzufügen
        all_features = pd.concat([all_features, extracted_features], ignore_index=True)
        done += 1
    
    for file_path in flatten_file_paths:
        if "measurements" not in file_path:
            continue
        
        all_measurements = pd.concat(
            [all_measurements, pd.read_csv(file_path)], ignore_index=True
        )
    
    all_features_labeled = all_features.copy()
    all_features_labeled = all_features_labeled.merge(
        all_measurements[["dataFile", "material"]], on="dataFile", how="left"
    )
    
    all_features_labeled.to_csv(output_file, index=False)
    print(f"Saved results for {folder_name} to {output_file}")